In [1]:
!pip install pytorch-lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 931.7/931.7 kB 43.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-

In [2]:
#@title load packages

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T


from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from pytorch_lightning.metrics.functional import accuracy

ModuleNotFoundError: No module named 'pytorch_lightning.metrics'

In [ ]:
#@title Load data

data_transforms = T.Compose([T.ToTensor(), T.Normalize((0.1307,), (0.3081,))])

mnist_training = MNIST(root='sample_data/', train=True, download=True,
                       transform=data_transforms)
mnist_test = MNIST(root='sample_data/', train=False, download=True,
                   transform=data_transforms)


In [ ]:
#@title create dataloader

batch_size = 500

training_generator = DataLoader(mnist_training, batch_size=batch_size, shuffle=True)
test_generator = DataLoader(mnist_test, batch_size=batch_size, shuffle=False)


In [ ]:
#@title define model

class MyModel(nn.Module):
  def __init__(self):
    super(MyModel, self).__init__()

    self.conv1 = nn.Conv2d(1, 32, 3)
    self.flatten = nn.Flatten()
    self.d1 = nn.Linear(32*26*26, 128)
    self.d2 = nn.Linear(128, 10)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.flatten(x)
    x = F.relu(self.d1(x))
    return self.d2(x)


model = MyModel().to('cuda')

In [ ]:
#@title define loss, optimizer and metrics

loss_object = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())


In [ ]:
#@title define training step

def train_step(images, labels):

    model.train()
    optimizer.zero_grad()

    predictions = model(images)
    loss = loss_object(predictions, labels)
    loss.backward()

    optimizer.step()

    return loss.item()


In [ ]:
#@title define test step

def test_step(images, labels):

    model.eval()

    predictions = F.softmax(model(images), dim=1)
    acc = accuracy(predictions, labels)

    return acc.item()


In [ ]:
#@title training loop

EPOCHS = 10


for epoch in range(EPOCHS):

  for images, labels in training_generator:
      train_loss = train_step(images.to('cuda'), labels.to('cuda'))

  for test_images, test_labels in test_generator:
      test_accuracy = test_step(test_images.to('cuda'), test_labels.to('cuda'))

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss}, '
    f'Test Accuracy: {test_accuracy * 100}'
  )